# 1梯度   
连结一个多元函数对其所有变量的偏导数，以得到该函数的梯度（gradient）向量。   
具体而言，设函数f：Rn—R的输入是一个
n维向量，但输出是一个标量。 函数fx相对于x的梯度是一个包含n个偏导数的向量:

其中 $\nabla_x f(x)$ 通常在没有歧义时被 $\nabla f(x)$ 取代。

假设 $x$ 为 $n$ 维向量，在微分多元函数时经常使用以下规则：

1. **规则 1**: 对于所有 $A \in \mathbb{R}^{m \times n}$，都有  
   $$\nabla_x (A x) = A^T$$  
   **注**: 此处存在笔误，正确结果应为 $(A + A^T)x$（当 $A$ 对称时退化为 $2Ax$）。
2. **规则 2**: 对于所有 $A \in \mathbb{R}^{n \times m}$，都有  
   $$\nabla_x (x^T A) = A$$
3. **规则 3**: 对于所有 $A \in \mathbb{R}^{n \times n}$，都有  
   $$\nabla_x (x^T A x) = (A + A^T)x$$
4. **规则 4**: 向量的平方范数  
   $$\nabla_x \|x\|^2 = \nabla_x (x^T x) = 2x$$
---
同样，对于任何矩阵 $X$，都有：  
$$\nabla_X \|X\|_F^2 = 2X$$  

# 2自动微分

In [3]:
import torch

In [4]:
x = torch.arange(4.0)
#在我们计算关于x的梯度之前，需要一个地方来存储梯度。 重要的是，我们不会在每次对一个参数求导时都分配新的内存。
# 因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。 
# 注意，一个标量函数关于向量的梯度是向量，并且与具有相同的形状。

tensor([0., 1., 2., 3.])

在PyTorch中，`x.requires_grad_(True)`用于启用张量`x`的梯度跟踪功能。以下是其作用及底层机制的详细说明：
### **1. 核心作用**
- **启用梯度跟踪**：设置`x.requires_grad = True`后，PyTorch的Autograd系统会开始追踪所有涉及`x`的操作，构建动态计算图。
- **自动求导的基础**：只有被标记为需要梯度的张量，才能在反向传播时自动计算其梯度（通过`.backward()`方法）。
### **2. 底层机制**
#### **(1) 设置标志位**
- **内部属性**：`requires_grad`是张量的一个布尔属性，默认值为`False`。
- **就地修改**：`x.requires_grad_(True)`是一个就地（in-place）操作，直接修改张量`x`的`requires_grad`属性为`True`。
#### **(2) 构建计算图**
- **操作记录**：当`x`参与任何数学运算（如加减乘除、矩阵乘法等）时，PyTorch会记录这些操作，并生成一个动态计算图。
- **依赖关系**：计算图记录了从输入（`x`）到输出（如损失函数）的所有中间步骤，为反向传播提供路径。
#### **(3) 梯度计算**
- **反向传播触发**：调用`.backward()`时，Autograd会根据计算图，从输出向输入反向传播，利用链式法则计算梯度。
- **梯度存储**：计算出的梯度会存储在张量的`.grad`属性中（如`x.grad`）。
---
#### **(2) 禁用梯度跟踪**
```python
x = torch.tensor([1.0], requires_grad=True)
with torch.no_grad():
    y = x * 2  # y.requires_grad = False
y.backward()  # 报错：y无梯度跟踪
```
### **6. 总结**
- **核心功能**：`x.requires_grad_(True)`激活梯度跟踪，是PyTorch自动求导的基础。
- **底层实现**：通过设置标志位和动态构建计算图实现。

In [32]:
test=(torch.tensor([1,2,3],dtype=float)).requires_grad_(True)
test
##当启用梯度追踪的时候 张量里会有一个属性requires_grad=True 可以访问
test.requires_grad

True

In [5]:
#启用torch的梯度跟踪
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

In [6]:
##定义一个fx 因变量为y
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [7]:
#接下来，通过调用反向传播函数来自动计算y关于x每个分量的梯度，并打印这些梯度。
y.backward()
x.grad

计算图不会累计，每次前向传播时旧图会被释放  
梯度会累积：当需要计算x的另一个函数的时候 需要清除梯度，如果不清零会影响新的关于x的函数梯度计算

In [9]:
#清除梯度
##带下划线的代表就地操作 无zero
x.grad.zero_()

tensor([0., 0., 0., 0.])

# 分离计算

In [15]:
#例如，假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。 想象一下，我们想计算z关于x的梯度，但由于某种原因，希望将y视为一个常数， 并且只考虑到x在y被计算后发挥的作用。
#这里可以分离y来返回一个新变量u，该变量与y具有相同的值， 但丢弃计算图中如何计算y的任何信息。 
# 换句话说，梯度不会向后流经u到x。 因此，下面的反向传播函数计算z=u*x关于x的偏导数，同时将u作为常数处理， 而不是z=x*x*x关于x的偏导数。
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

###使用y.detach() 对y的进行分离

tensor([True, True, True, True])

# 使用自动微分的一个好处是： 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。 在下面的代码中，while循环的迭代次数和if语句的结果都取决于输入a的值。